## Importing all the necessary dependencies

In [1]:
import pandas as pd 
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

## Reading the CSV files

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4629,300000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 27, ""nam...",http://someguywhokillspeople.com/,80468,[],en,Some Guy Who Kills People,A former mental patient's repressed anger reac...,2.733725,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2011-04-14,0,97.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Your number is up.,Some Guy Who Kills People,5.7,45
2693,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10402, ""n...",NaN,12637,"[{""id"": 242, ""name"": ""new york""}, {""id"": 1416,...",en,"New York, New York",An egotistical saxophone player and a young si...,3.876242,"[{""name"": ""United Artists"", ""id"": 60}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1977-06-21,0,155.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"New York, New York",6.1,67
1106,44000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,1369,"[{""id"": 279, ""name"": ""usa""}, {""id"": 422, ""name...",en,Rambo: First Blood Part II,John Rambo is released from prison by the gove...,38.653969,"[{""name"": ""TriStar Pictures"", ""id"": 559}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1985-05-21,300400432,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"What most people call hell, he calls home.",Rambo: First Blood Part II,6.3,864
4035,2500000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 27, ""nam...",NaN,254472,"[{""id"": 158091, ""name"": ""giant insect""}]",de,Stung,A fancy garden party turns into upper class pr...,4.270489,"[{""name"": ""Rat Pack Filmproduktion"", ""id"": 575...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-07-03,0,87.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Get inside, get away, just don't get stung",Stung,4.9,63
453,0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",NaN,12096,"[{""id"": 642, ""name"": ""robbery""}, {""id"": 5340, ...",en,The Pink Panther,When the coach of the France soccer team is ki...,25.450534,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2006-01-18,0,93.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Pardon His French.,The Pink Panther,5.6,550


In [4]:
credits.sample(5)

,movie_id,title,cast,crew
4016,401,Garden State,"[{""cast_id"": 1, ""character"": ""Andrew Largeman""...","[{""credit_id"": ""52fe423fc3a36847f800f449"", ""de..."
871,8046,Gigli,"[{""cast_id"": 9, ""character"": ""Larry Gigli"", ""c...","[{""credit_id"": ""562748bb92514137cf001a55"", ""de..."
2451,10760,Sydney White,"[{""cast_id"": 12, ""character"": ""Sydney White"", ...","[{""credit_id"": ""52fe43b19251416c7501a685"", ""de..."
2473,5038,Vicky Cristina Barcelona,"[{""cast_id"": 3, ""character"": ""Cristina"", ""cred...","[{""credit_id"": ""52fe43eec3a36847f80792ad"", ""de..."
1129,2637,The Mothman Prophecies,"[{""cast_id"": 1, ""character"": ""John Klein"", ""cr...","[{""credit_id"": ""52fe4361c3a36847f804fe1f"", ""de..."


In [5]:
movies = movies.merge(credits, on='title') 

In [6]:
movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
4564,0,"[{""id"": 10402, ""name"": ""Music""}, {""id"": 27, ""n...",http://www.thedevilscarnival.com/,285743,"[{""id"": 3473, ""name"": ""carnival""}, {""id"": 4344...",en,Alleluia! The Devil's Carnival,The Devil's Carnival: Alleluia! is the second ...,0.674398,"[{""name"": ""Limb from Limb Pictures"", ""id"": 590...",...,0.0,[],Released,Hell ain't got a prayer.,Alleluia! The Devil's Carnival,6.0,2,285743,"[{""cast_id"": 11, ""character"": ""Lucifer"", ""cred...","[{""credit_id"": ""53e40a4f0e0a262b8f0050c4"", ""de..."
2632,15000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",http://www.foxsearchlight.com/neverletmego/,42188,"[{""id"": 634, ""name"": ""soul""}, {""id"": 818, ""nam...",en,Never Let Me Go,"As children, Kathy, Ruth, and Tommy spend thei...",30.983397,"[{""name"": ""Fox Searchlight Pictures"", ""id"": 43...",...,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,These students have everything they need. Exce...,Never Let Me Go,6.8,737,42188,"[{""cast_id"": 6, ""character"": ""Kathy H."", ""cred...","[{""credit_id"": ""52fe45f3c3a36847f80e3c91"", ""de..."
1469,34000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",NaN,10436,"[{""id"": 213, ""name"": ""upper class""}, {""id"": 24...",en,The Age of Innocence,Tale of 19th century New York high society in ...,9.133747,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",...,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,In a world of tradition. In an age of innocenc...,The Age of Innocence,7.0,169,10436,"[{""cast_id"": 1, ""character"": ""Newland Archer"",...","[{""credit_id"": ""52fe43709251416c750106c5"", ""de..."
4255,0,"[{""id"": 99, ""name"": ""Documentary""}]",http://www.walmartmovie.com/,29595,"[{""id"": 279, ""name"": ""usa""}, {""id"": 592, ""name...",en,Wal-Mart: The High Cost of Low Price,This documentary takes the viewer on a deeply ...,1.404175,"[{""name"": ""Brave New Films"", ""id"": 4971}]",...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Wal-Mart: The High Cost of Low Price,6.9,24,29595,"[{""cast_id"": 8, ""character"": ""Himself - Presid...","[{""credit_id"": ""52fe45ebc3a368484e0767f7"", ""de..."
1753,27000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",NaN,9583,"[{""id"": 2908, ""name"": ""secret society""}, {""id""...",en,Divine Secrets of the Ya-Ya Sisterhood,A mother and daughter dispute is resolved by t...,5.576937,"[{""name"": ""All Girl Productions"", ""id"": 10535}...",...,116.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Secret Is Out.,Divine Secrets of the Ya-Ya Sisterhood,5.6,65,9583,"[{""cast_id"": 1, ""character"": ""Siddalee 'Sidda'...","[{""credit_id"": ""555dd65ec3a36868c9001463"", ""de..."


In [7]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [8]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
1974,10313,The Men Who Stare at Goats,A reporter in Iraq might just have the story o...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10752, ""...","[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...","[{""cast_id"": 3, ""character"": ""Lyn Cassady"", ""c...","[{""credit_id"": ""5989ab3a925141370600675e"", ""de..."
3275,44009,Another Year,Mike Leigh’s much praised 2010 tragicomical dr...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 931, ""name"": ""jealousy""}, {""id"": 1918,...","[{""cast_id"": 10, ""character"": ""Tom"", ""credit_i...","[{""credit_id"": ""5353c5a70e0a265603004b63"", ""de..."
4779,2292,Clerks,Convenience and video store clerks Dante and R...,"[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 1361, ""name"": ""salesclerk""}, {""id"": 30...","[{""cast_id"": 7, ""character"": ""Dante Hicks"", ""c...","[{""credit_id"": ""52fe434ac3a36847f8049253"", ""de..."
2993,8922,Jeepers Creepers,A college-age brother and sister get more than...,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...","[{""id"": 1583, ""name"": ""mass murder""}, {""id"": 4...","[{""cast_id"": 10, ""character"": ""Patricia 'Trish...","[{""credit_id"": ""52fe44c8c3a36847f80a9c27"", ""de..."
2628,6552,Idle Hands,Anton is a cheerful but exceedingly non-ambiti...,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 35, ""n...","[{""id"": 13130, ""name"": ""teenager""}, {""id"": 155...","[{""cast_id"": 1, ""character"": ""Anton"", ""credit_...","[{""credit_id"": ""52fe445ac3a36847f8090f0d"", ""de..."


## Data Preprocessing

In [9]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [10]:
movies.dropna(inplace=True)

In [11]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [12]:
movies.duplicated().sum()

0

In [13]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [14]:
def create_genre_list_of_movie(movie_genre_object_array):
    
    genre_name_list = []
    
    for i in ast.literal_eval(movie_genre_object_array):
        
        
        genre_name_list.append(i['name'])
        
        
    return genre_name_list

In [15]:
movies['genres'] = movies['genres'].apply(create_genre_list_of_movie)

In [16]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
1162,37799,The Social Network,"On a fall night in 2003, Harvard undergrad and...",[Drama],"[{""id"": 2157, ""name"": ""hacker""}, {""id"": 12361,...","[{""cast_id"": 7, ""character"": ""Mark Zuckerberg""...","[{""credit_id"": ""52fe466f9251416c9105471d"", ""de..."
610,188207,The Legend of Hercules,"In Ancient Greece 1200 B.C., a queen succumbs ...","[Action, Adventure]","[{""id"": 2035, ""name"": ""mythology""}, {""id"": 898...","[{""cast_id"": 1, ""character"": ""Hercules"", ""cred...","[{""credit_id"": ""52fe4d329251416c75133441"", ""de..."
2680,12227,White Fang,Jack London's classic adventure story about th...,"[Action, Adventure, Drama, Family]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 14, ""character"": ""Alex Larson"", ""...","[{""credit_id"": ""52fe44ce9251416c750418cb"", ""de..."
3894,41436,Paranormal Activity 2,Just as Dan and Kristi welcome a newborn baby ...,"[Horror, Thriller]","[{""id"": 10112, ""name"": ""ouija board""}, {""id"": ...","[{""cast_id"": 3, ""character"": ""Katie"", ""credit_...","[{""credit_id"": ""535d00bdc3a368227300048e"", ""de..."
3755,5925,The Great Escape,"The Nazis, exasperated at the number of escape...","[Adventure, Drama, History, Thriller, War]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 1, ""character"": ""Captain Hilts"", ...","[{""credit_id"": ""52fe442fc3a36847f8087235"", ""de..."


In [17]:
def create_keyword_list_of_movie(movie_keyword_object_array):
    
    keyword_name_list = []
    
    for i in ast.literal_eval(movie_keyword_object_array):
        
        keyword_name_list.append(i['name'])
        
        
    return keyword_name_list

In [18]:
movies['keywords'] = movies['keywords'].apply(create_keyword_list_of_movie)

In [19]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
372,1535,Spy Game,Veteran spy Nathan Muir is on the verge of ret...,"[Action, Crime, Thriller]","[spy, china, cia, cold war]","[{""cast_id"": 2, ""character"": ""Nathan D. Muir"",...","[{""credit_id"": ""559fd2d8c3a3680810005604"", ""de..."
2012,354110,Forsaken,John Henry returns to his hometown in hopes of...,"[Western, Drama]",[],"[{""cast_id"": 1, ""character"": ""Mary-Alice Watso...","[{""credit_id"": ""576bbf53c3a3683706002b19"", ""de..."
669,189,Sin City: A Dame to Kill For,Some of Sin City's most hard-boiled citizens c...,"[Crime, Thriller]","[detective, dystopia, dominatrix, murder, susp...","[{""cast_id"": 5, ""character"": ""Marv"", ""credit_i...","[{""credit_id"": ""52fe4224c3a36847f800743f"", ""de..."
3967,192210,Open Road,"Angie, a young Brazilian artist, abandons her ...","[Action, Drama, Thriller]","[drug dealer, police, killer, illegal drugs, d...","[{""cast_id"": 2, ""character"": ""Angie"", ""credit_...","[{""credit_id"": ""52fe4c9e9251416c910fab5d"", ""de..."
1491,12596,Pushing Tin,"Two air traffic controllers (John Cusack, Bill...","[Comedy, Drama]","[competition, airport, air controller, adversary]","[{""cast_id"": 1, ""character"": ""Nick Falzone"", ""...","[{""credit_id"": ""52fe45039251416c75048bc1"", ""de..."


In [20]:
def create_top_three_cast_list_of_movie(cast_object_array):
    
    top_three_cast_name_list = []
    
    counter = 0
    
    for i in ast.literal_eval(cast_object_array):
        
        if counter != 3:
            
            top_three_cast_name_list.append(i['name'])
            
            counter += 1
        
        else:
            
            break

        
    return top_three_cast_name_list

In [21]:
movies['cast'] = movies['cast'].apply(create_top_three_cast_list_of_movie)

In [22]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
461,2157,Lost in Space,The prospects for continuing life on Earth in ...,"[Adventure, Family, Science Fiction]","[time travel, sabotage, deep space explorer]","[Gary Oldman, William Hurt, Matt LeBlanc]","[{""credit_id"": ""52fe433bc3a36847f80448b9"", ""de..."
2155,38303,You Again,History -- make that high school -- may repeat...,"[Comedy, Romance]","[brother sister relationship, marriage, bully,...","[Kristen Bell, Sigourney Weaver, Betty White]","[{""credit_id"": ""52fe46a89251416c9105b75b"", ""de..."
3323,12569,Extract,The owner of a factory that produces flavor ex...,[Comedy],"[infidelity, con man, thief, independent film,...","[Jason Bateman, Mila Kunis, Kristen Wiig]","[{""credit_id"": ""57c8a895c3a3685c020006eb"", ""de..."
876,11456,Domestic Disturbance,A divorced father discovers that his 12-year-o...,"[Mystery, Thriller, Crime]","[menace, adoption, dangerous, adoptive father,...","[John Travolta, Vince Vaughn, Teri Polo]","[{""credit_id"": ""52fe44439251416c7502e9a3"", ""de..."
3760,13408,The Wash,"With the rent due and his car booted, Sean (Dr...","[Action, Comedy]","[sex, fight, music, illegal drug, black americ...","[Dr. Dre, Snoop Dogg, DJ Pooh]","[{""credit_id"": ""52fe45659251416c75054fb9"", ""de..."


In [23]:
def crew_list_of_movie(crew_object_array):
    
    crew_name_list = []
    
    for i in ast.literal_eval(crew_object_array):
        
        if i['job'] == 'Director':
            
            crew_name_list.append(i['name'])
            
            break
            
            
    return crew_name_list

In [24]:
movies['crew'] = movies['crew'].apply(crew_list_of_movie)

In [25]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
3538,5528,The Chorus,"Set in 1940's France, a new teacher at a schoo...",[Drama],"[penalty, choir, diary, musical, boy's choir, ...","[Gérard Jugnot, François Berléand, Kad Merad]",[Christophe Barratier]
1165,4234,Scream 3,"A murdering spree begins to happen again, this...","[Horror, Mystery]","[mask, metal detector, film director, ex-cop, ...","[David Arquette, Neve Campbell, Courteney Cox]",[Wes Craven]
2805,23631,Machete,After being set-up and betrayed by the man who...,"[Action, Comedy, Thriller]","[illegal immigration, immigration law, machete...","[Danny Trejo, Michelle Rodriguez, Jessica Alba]",[Robert Rodriguez]
424,27578,The Expendables,Barney Ross leads a band of highly skilled mer...,"[Thriller, Adventure, Action]","[tattoo, martial arts, sniper, island, mercena...","[Sylvester Stallone, Jason Statham, Dolph Lund...",[Sylvester Stallone]
2531,1165,The Queen,The Queen is an intimate behind the scenes gli...,[Drama],"[upper class, loss of mother, great britain, s...","[Helen Mirren, Michael Sheen, James Cromwell]",[Stephen Frears]


In [26]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [27]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
3813,114635,The Helpers,"[Seven, friends, from, Sacramento,, California...",[Horror],[],"[Kristen Quintrall, Denyce Lawton, JoJo Wright]",[Chris Stokes]
3372,57825,Bride & Prejudice,"[A, Bollywood, update, of, Jane, Austen's, cla...","[Drama, Comedy, Music, Romance]","[indian lead, bollywood, modern day adaptation...","[Aishwarya Rai Bachchan, Martin Henderson, Nav...",[Gurinder Chadha]
3680,77949,The Awakening,"[1921, England, is, overwhelmed, by, the, loss...","[Horror, Thriller]","[suicide, england, classroom, investigation, s...","[Rebecca Hall, Dominic West, Imelda Staunton]",[Nick Murphy ]
2794,9504,Glengarry Glen Ross,"[Glengarry, Glen, Ross,, follows, the, lives, ...","[Crime, Drama, Mystery]","[robbery, office, shop, estate agent, company,...","[Al Pacino, Jack Lemmon, Alec Baldwin]",[James Foley]
3188,15568,Gracie,"[This, is, the, story, of, a, teenager, named,...",[Drama],[],"[Jesse Lee Soffer, Christopher Shand, Carly Sc...",[Davis Guggenheim]


In [28]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ', '') for i in x])

In [29]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
1013,6415,Three Kings,"[A, group, of, American, soldiers, stationed, ...","[Action, Adventure, Comedy]","[gold, gulfwar, threekings, iraq, kuwait, resc...","[GeorgeClooney, MarkWahlberg, IceCube]",[DavidO.Russell]
4269,312791,GirlHouse,"[In, an, attempt, to, make, some, extra, cash,...","[Horror, Thriller]","[femalenudity, sex, shower, sauna, nudity, sec...","[AliCobrin, ErinAgostino, JamesThomas]",[JonKnautz]
1495,50647,Wanderlust,"[Rattled, by, sudden, unemployment,, a, Manhat...",[Comedy],"[hippie, commune, nudism, nudeprotest, aftercr...","[PaulRudd, JenniferAniston, JustinTheroux]",[DavidWain]
3609,192141,No Escape,"[In, their, new, overseas, home,, an, American...","[Action, Thriller]","[hotel, revolution, raceagainsttime, engineer,...","[OwenWilson, LakeBell, PierceBrosnan]",[JohnErickDowdle]
4204,25312,Yes,"[She, is, a, scientist., He, is, a, Lebanese, ...","[Drama, Romance]",[womandirector],"[JoanAllen, SimonAbkarian, SamNeill]",[SallyPotter]


In [30]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [31]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew,tags
4150,18616,No Man's Land: The Rise of Reeker,"[A, sheriff, and, his, son, who, are, tracking...","[Horror, Thriller]","[sheriff, robber, burnedalive, gore, serialkil...","[StephenMartines, ValerieCruz, MirceaMonroe]",[DavePayne],"[A, sheriff, and, his, son, who, are, tracking..."
3929,10557,East Is East,"[In, 1971, Salford, fish-and-chip, shop, owner...",[Comedy],"[islam, hippie, fathersonrelationship, muslim,...","[OmPuri, LindaBassett, IanAspinall]",[DamienO'Donnell],"[In, 1971, Salford, fish-and-chip, shop, owner..."
4539,172533,Drinking Buddies,"[Weekend, trips,, office, parties,, late, nigh...","[Romance, Drama]","[malefemalerelationship, co-worker, relationsh...","[OliviaWilde, JakeJohnson, AnnaKendrick]",[JoeSwanberg],"[Weekend, trips,, office, parties,, late, nigh..."
2493,3877,Things We Lost in the Fire,"[A, recent, widow, invites, her, husband's, tr...",[Drama],"[sadness, lossoffather, junkie, luxury, heroin...","[HalleBerry, BeniciodelToro, DavidDuchovny]",[SusanneBier],"[A, recent, widow, invites, her, husband's, tr..."
3348,8390,"Definitely, Maybe","[When, Will, decides, to, tell, his, daughter,...","[Comedy, Romance]","[lovesickness, loveofone'slife, narration, lov...","[RyanReynolds, AbigailBreslin, ElizabethBanks]",[AdamBrooks],"[When, Will, decides, to, tell, his, daughter,..."


In [32]:
movies = movies[['movie_id', 'title', 'tags']]

In [33]:
movies

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [34]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [35]:
movies.sample(5)

,movie_id,title,tags
1872,9655,She's the Man,Viola Johnson is in a real jam. Complications ...
4493,51955,Hav Plenty,Lee Plenty is an almost broke would-be novelis...
4708,408429,Perfect Cowboy,Two gay fathers of a straight country western ...
2946,11184,Kinsey,Kinsey is a portrait of researcher Alfred Kins...
996,109491,Beautiful Creatures,Ethan Wate just wants to get to know Lena Duch...


In [36]:
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [37]:
movies.sample(5)

,movie_id,title,tags
3765,18713,The Jerky Boys,when two unemployed telephone pranksters decid...
4525,41144,To Save A Life,jake taylor has everything. he has a beautiful...
1817,8681,Taken,"while vacationing with a friend in paris, an a..."
2752,136835,No Good Deed,"terri is a devoted wife and mother of two, liv..."
1092,10858,Nixon,an all-star cast powers this epic look at amer...


## Applying Vectorization and Cosine Similarity

In [38]:
ps = PorterStemmer()

In [39]:
def stem_words(text):
    
    y = []
    
    for i in text.split():
        
        y.append(ps.stem(i))
        
    return " ".join(y) 

In [40]:
movies['tags'] = movies['tags'].apply(stem_words)

In [41]:
movies.sample(5)

,movie_id,title,tags
3640,245916,Kill the Messenger,a report becom the target of a viciou smear ca...
4323,157422,The Wicked Within,after a year pass sinc the sudden death of a c...
4435,271185,Ask Me Anything,"beautiful, wild, funny, and lost, kati kampenf..."
4232,127918,The Gatekeepers,"in an unpreced and candid seri of interviews, ..."
1583,4133,Blow,a boy name georg jung grow up in a struggl fam...


In [42]:
vectorizer = CountVectorizer(max_features=5000, stop_words='english')

In [43]:
vector_of_tags = vectorizer.fit_transform(movies['tags']).toarray()

In [44]:
similarity_matrix = cosine_similarity(vector_of_tags)

## Recommending top 5 movies for a given movie name

In [45]:
def recommend_top_5_similar_movies_for_the_given_movie_input(movie_name):
    
    index_of_the_movie = movies[movies['title'] == movie_name].index[0]
    
    all_distances = similarity_matrix[index_of_the_movie]
    
    top_five_movies = sorted(list(enumerate(all_distances)), reverse=True, key=lambda x:x[1])[1:6] 

    for i in top_five_movies:
        
        print(movies.iloc[i[0]].title)
    

In [46]:
recommend_top_5_similar_movies_for_the_given_movie_input('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


## Creating a pickle file out of the cleaned dataset and the similarity matrix

In [47]:
pickle.dump(movies.to_dict(), open('cleaned_movies_dataset_dictionary.pkl', 'wb'))

In [48]:
pickle.dump(similarity_matrix, open('similarity_matrix.pkl', 'wb'))